In [10]:
import cv2 as cv
import numpy as np
import os

### **Generalización para múltiples detecciones de un mismo template**

Abarca items 1, 2 y 3.

In [11]:
# --- Ruta de la carpeta con las imágenes ---
directorio = "./images"

# --- Mejores escalas encontradas ---
scales = {
    'COCA-COLA-LOGO.jpg': 0.31,
    'coca_multi.png': 4.03,
    'coca_logo_1.png': 2.3,
    'coca_retro_1.png': 0.76,
    'logo_1.png': 1.34,
    'coca_logo_2.png': 1.77,
    'coca_retro_2.png': 2.49
}

# --- Lista para guardar las imágenes ---
imagenes = []

# --- Recorremos los archivos del directorio ---
for archivo in os.listdir(directorio):
    ruta_completa = os.path.join(directorio, archivo)

    # --- Verificamos que sea un archivo y termine en una extensión de imagen ---
    if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg')):
        img = cv.imread(ruta_completa)
        scale = scales[archivo]
        img = cv.resize(img, None, fx=scale, fy=scale)
        if img is not None:
            imagenes.append(img)
        else:
            print(f"No se pudo leer la imagen: {archivo}")

print(f"Se leyeron {len(imagenes)} imágenes")

Se leyeron 7 imágenes


In [12]:
def preproc_img(img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img_blur = cv.GaussianBlur(img_gray, (5,5), 0)
    img_edges = cv.Canny(img_blur, 50, 150)
    return img_edges

In [ ]:
# --- Cargar template ---
template = cv.imread("./template/pattern.png")

# --- Preprocesar template ---
template_preproc = preproc_img(template)

for img in imagenes:
    h, w = template_preproc.shape

    # Preprocesar imagen escalada
    img_preproc = preproc_img(img)

    # Matching
    result = cv.matchTemplate(img_preproc, template_preproc, cv.TM_CCOEFF_NORMED)

    # Los matches encontrados estaban por encima del 0.07
    threshold = 0.07

    loc = np.where(result >= threshold)
    rects = []

    for pt in zip(*loc[::-1]):
        rects.append([pt[0], pt[1], w, h])

    # Agrupar rectángulos que se solapan
    rects, weights = cv.groupRectangles(rects, groupThreshold=1, eps=0.5)

    for (x, y, w, h) in rects:
        cv.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Obtener mejor resultado
    _, max_val, _, max_loc = cv.minMaxLoc(result)
    print(f"Mejor coincidencia: {max_val:.4f}")
    cv.imshow("Mejor resultado", img)
    cv.waitKey(0)
    cv.destroyAllWindows()

Mejor coincidencia: 0.0925
Mejor coincidencia: 0.1098
Mejor coincidencia: 0.2993
Mejor coincidencia: 0.0902
Mejor coincidencia: 0.1168
Mejor coincidencia: 0.0832
Mejor coincidencia: 0.1687


: 

### **Conclusiones**

Se realizó un resize de las imágenes objetivos realizando primero una serie de pruebas para encontrar la escala que tuviera el mejor resultado para un match del template, obteniendo las siguientes escalas para cada imagen:

- COCA-COLA-LOGO.jpg: 0.31
- coca_multi.png: 4.03
- coca_logo_1.png: 2.3
- coca_retro_1.png: 0.76
- logo_1.png: 1.34
- coca_logo_2.png: 1.77
- coca_retro_2.png: 2.49

Luego se realizó un preprocesamiento de tanto el template como las imágenes usando primero conversión a escala de grises, desenfoque Gaussiano y detección de bordes mediante método de Canny.

Finalmente se unificó el código para agrupar todos recuadros de los matches de manera de no tener superposiciones y se mostró la mejor correlación obtenida, que tuviera sentido al ver el recuadro en la imagen. Esto se menciona porque aunque se obtuvieron correlaciones más altas en diferentes pruebas, por ejemplo al usar pirámides sobre los templates, al mostrar el recuadro el match no era correcto con el logo deseado.